In [14]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [15]:
url='https://www.meteo-paris.com/ile-de-france/previsions'

In [16]:
r=requests.get(url)
status = r.status_code

if status==200:
    print('OK')
elif status==404:
    print("Error")
elif status==403:
    print("Access forbidden")
elif status==500:
    print("Internal server error")

OK


In [17]:
s = BeautifulSoup(r.content,'html.parser')

In [18]:
div_pred_temp = []
for text in s.find_all('div',{'class':'forecast-line__pictos--temp'}):
    b = text
    div_pred_temp.append(b)
    
print(div_pred_temp)

[<div class="forecast-line__pictos--temp bg-plus-6">6</div>, <div class="forecast-line__pictos--temp bg-plus-11">11</div>, <div class="forecast-line__pictos--temp bg-plus-3">3</div>, <div class="forecast-line__pictos--temp bg-plus-14">14</div>, <div class="forecast-line__pictos--temp bg-plus-5">5</div>, <div class="forecast-line__pictos--temp bg-plus-8">8</div>, <div class="forecast-line__pictos--temp bg-minus-1">-1</div>, <div class="forecast-line__pictos--temp bg-plus-6">6</div>, <div class="forecast-line__pictos--temp bg-minus-1">-1</div>, <div class="forecast-line__pictos--temp bg-plus-8">8</div>, <div class="forecast-line__pictos--temp bg-minus-1">-1</div>, <div class="forecast-line__pictos--temp bg-plus-12">12</div>, <div class="forecast-line__pictos--temp bg-plus-4">4</div>, <div class="forecast-line__pictos--temp bg-plus-16">16</div>, <div class="forecast-line__pictos--temp bg-plus-10">10</div>, <div class="forecast-line__pictos--temp bg-plus-17">17</div>, <div class="forecast-

In [19]:
from datetime import date

today = date.today().strftime("%d/%m/%Y")
pred_temp = [pt.get_text() for pt in div_pred_temp][2:]
pred_temp.insert(0,today)
print(pred_temp)

['03/04/2021', '3', '14', '5', '8', '-1', '6', '-1', '8', '-1', '12', '4', '16', '10', '17', '8', '12']


In [20]:
from io import StringIO

url_data='https://static.meteo-paris.com/station/downld02.txt'
r_data=requests.get(url_data)

aux = StringIO(r_data.text)
col_names = ['Date','Time','Temp_Out','Hi_Temp','Low_Temp','Out_Hum','Dew_Pt','Wind_Speed',
            'Wind_Dir','Wind_Run','Hi_Speed','Hi_Dir','Wind_Chill','Heat_Index','THW_Index','Bar',
            'Rain','Rain_Rate','Heat_DD','Cool_DD','In_Temp','In_Hum','In_Dew','In_Heat',
             'In_EMC','In_Air_Dens','Temp_2nd','Wind_Samp','Wind_TX','ISS_Recept','Arc_Int']
df_list = pd.read_csv(aux,skiprows=3,delimiter="\s+",header=None,names=col_names)
df_list.head()

,Date,Time,Temp_Out,Hi_Temp,Low_Temp,Out_Hum,Dew_Pt,Wind_Speed,Wind_Dir,Wind_Run,...,In_Hum,In_Dew,In_Heat,In_EMC,In_Air_Dens,Temp_2nd,Wind_Samp,Wind_TX,ISS_Recept,Arc_Int
0,2/04/21,0:01,14.8,14.8,14.8,73,10.0,8.0,NNE,0.13,...,35,10.5,26.5,6.8,1.1715,---,22,1,95.7,1
1,2/04/21,0:02,14.8,14.8,14.8,73,10.0,8.0,NW,0.13,...,35,10.5,26.5,6.8,1.1714,---,23,1,100.0,1
2,2/04/21,0:03,14.8,14.8,14.8,73,10.0,9.7,NNW,0.16,...,35,10.5,26.5,6.8,1.1715,---,24,1,100.0,1
3,2/04/21,0:04,14.8,14.8,14.7,73,10.0,14.5,NNW,0.24,...,35,10.5,26.5,6.8,1.1715,---,22,1,95.7,1
4,2/04/21,0:05,14.7,14.8,14.7,73,9.9,11.3,NNW,0.19,...,35,10.5,26.5,6.8,1.1715,---,23,1,100.0,1


In [21]:
df_list[df_list['Date']=='28/03/21']['Temp_Out'].max()

nan

In [22]:
from datetime import datetime,timedelta

yesterday = datetime.now() - timedelta(1)
datetime.strftime(yesterday, '%Y-%m-%d')

datetime.strptime('28/03/21 0:02', '%d/%m/%y %H:%M')

datetime.datetime(2021, 3, 28, 0, 2)

In [23]:
#for d in df_list.iterrows():
    #datetime.strptime(d[1]['Date'], '%d/%m/%y')
    
pd.to_datetime(df_list['Date'] + ' ' + df_list['Time'])

#df_list['Date_format']=datetime.strptime(df_list['Date'], '%d/%m/%y')

0      2021-02-04 00:01:00
1      2021-02-04 00:02:00
2      2021-02-04 00:03:00
3      2021-02-04 00:04:00
4      2021-02-04 00:05:00
               ...        
2154   2021-03-04 11:55:00
2155   2021-03-04 11:56:00
2156   2021-03-04 11:57:00
2157   2021-03-04 11:58:00
2158   2021-03-04 11:59:00
Length: 2159, dtype: datetime64[ns]

In [24]:
### NOAA documentation
### https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted
### dataset='https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets'

In [92]:
#Most diffifcult part is to get the GHCND code for locations

def request_weather_arxiv(semester,year):
    dataset='GHCND'
    location="GHCND:FR000007150" #Paris Montsouris
    #location="GHCND:FRM00007149" #Paris Orly
    #location="GHCND:EI000003969" #Dublin Phoenix Park
    n_token='VMPMXiToPnVyavbJBWucazFqvkfkQnAD'

    if semester==1:
        s_date = str(year) + '-01-01'
        e_date = str(year) + '-06-30'
    elif semester==2:
        s_date = str(year) + '-07-01'
        e_date = str(year) + '-12-31'

    weathers_url="https://www.ncdc.noaa.gov/cdo-web/api/v2/data/"
    params='stationid=' + location + '&' 'datasetid=' + dataset + '&' + 'startdate=' + s_date + '&' + 'enddate=' + e_date + '&' + 'limit=1000' + '&' + 'units=metric'# + '&datatypeid=TMAX'
    noaa_token = {'token': n_token}

    req = requests.get(weathers_url, params=params, headers=noaa_token)
    df_req = pd.DataFrame(req.json()['results'])
    
#    print(df_req)
    
    pd.DataFrame(df_req,columns=['date','datatype','value'])\
      .pivot(index=['date'],columns='datatype',values='value')\
      .reset_index(drop=False)\
      .to_csv('ParisMontSouris_weather_arxiv.csv',index=False,mode='a',header=False)
    
#    pd.DataFrame(df_req,columns=['date','datatype','value'])\
#      .pivot(index=['date'],columns='datatype',values='value')\
#      .reset_index(drop=False)\
#      .drop(columns='TAVG')\
#      .to_csv('ParisMontSouris_weather_arxiv.csv',index=False,mode='a',header=False)
      
    #    return df_req

#df=request_weather_arxiv(1,2000)

for year in range(1980,2021):    
    request_weather_arxiv(1,year)
    request_weather_arxiv(2,year)
    print('Year {}: DONE'.format(year))

Year 1980: DONE
Year 1981: DONE
Year 1982: DONE
Year 1983: DONE
Year 1984: DONE
Year 1985: DONE
Year 1986: DONE
Year 1987: DONE
Year 1988: DONE
Year 1989: DONE
Year 1990: DONE
Year 1991: DONE


KeyboardInterrupt: 

In [70]:
df_arxiv=pd.DataFrame(df_req,columns=['date','datatype','value'])\
            .pivot(index=['date'],columns='datatype',values='value')\
            .reset_index(drop=False)\
            .drop(columns='TAVG')\

In [69]:
df_arxiv.to_csv('ParisMontSouris_weather_arxiv.csv',index=False,mode='a',header=False)

In [152]:
def request_weather_arxiv(datatype,year):
    dataset='GHCND'
    location="GHCND:FR000007150" #Paris Montsouris
    
    n_token='VMPMXiToPnVyavbJBWucazFqvkfkQnAD'

    s_date = str(year) + '-01-01'
    e_date = str(year) + '-12-31'

    weathers_url="https://www.ncdc.noaa.gov/cdo-web/api/v2/data/"
    params='stationid=' + location + '&' 'datasetid=' + dataset + '&' + 'startdate=' + s_date + '&' + 'enddate=' + e_date + '&' + 'limit=1000' + '&' + 'units=metric' + '&datatypeid=' + datatype
    noaa_token = {'token': n_token}

    req = requests.get(weathers_url, params=params, headers=noaa_token)
    #print(req.json())
    df_req = pd.DataFrame(req.json()['results'])
      
    return pd.DataFrame(df_req,columns=['date','value'])

for year in range(2004,2021):
    df_PRCP = request_weather_arxiv('PRCP',year)
    df_TMIN = request_weather_arxiv('TMIN',year)
    df_TMAX = request_weather_arxiv('TMAX',year)
    
    df_aux=pd.merge(df_TMIN,df_TMAX,how='outer',on='date',suffixes=('_TMIN','_TMAX'))
    
    pd.merge(df_aux,df_PRCP,how='outer',on='date',suffixes=('','_PRCP'))\
      .sort_values(by='date')\
      .to_csv('ParisMontSouris_weather_arxiv.csv',index=False,mode='a',header=False)
    
    print('Year {}: DONE'.format(year),end='\r')